### Import Libraries

In [3]:
#libraties
import numpy as np
import pandas as pd
from surprise import KNNBasic, SVD,SVDpp, CoClustering, Dataset, Reader,KNNBaseline
from IESEGRecSys import eval
from IESEGRecSys.model import ContentBased
from sklearn.model_selection import train_test_split
from surprise import BaselineOnly
from sklearn import linear_model

### Load Tabled

In [4]:
#Load tables with our metadata, train information and test data that we are going to predict at the end
metadata = pd.read_csv("./Data/metadata.csv")
sample_sub = pd.read_csv("./Data/sample_submission.csv")
test_Student = pd.read_csv("./Data/test_students.csv")
train = pd.read_csv("./Data/train.csv")


### Data peaking

In [5]:

metadata.head()


,asin,category,description,title,image,feature,main_cat,price
0,1612231977,"['Pet Supplies', 'Dogs', 'Health Supplies', 'H...",['Dr. Rexy hemp oil has powerful anti-inflamma...,DR.REXY Hemp Oil for Dogs and Cats - 100% Orga...,['https://images-na.ssl-images-amazon.com/imag...,['Made strictly with organic derived ingredien...,Amazon Home,$19.90
1,6162622851,"['Pet Supplies', 'Dogs', 'Flea & Tick Control'...",['Kills and repels fleas and ticks for 8 conti...,Bayer Seresto Flea and Tick Collar for Dogs,['https://images-na.ssl-images-amazon.com/imag...,['Veterinarian-recommended ea and tick prevent...,Pet Supplies,$37.99
2,B00000IRNW,"['Pet Supplies', 'Dogs', 'Toys', 'Balls']",['100 Rokenbok balls. 50 large blue balls and ...,Rokenbok ROK Balls,[],"['Modular- interacts with all Rokenbok', 'Roke...",Toys & Games,NaN
3,B00004T2WR,"['Pet Supplies', 'Dogs', 'Collars, Harnesses &...",['The Get Up \'n Go Discovery Center from Play...,Exclusive Playskool Electronic Activity Table,['https://images-na.ssl-images-amazon.com/imag...,"[""INTELLIGENT ANTI-INJURY CHIP: We always put ...",Pet Supplies,NaN
4,B00005MF9U,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",['LitterMaid LM900 self-cleaning cat litter bo...,LitterMaid LM900 Mega Self-Cleaning Litter Box,['https://images-na.ssl-images-amazon.com/imag...,['Automatically rakes waste into sealable cont...,Pet Supplies,NaN


In [6]:
sample_sub


,ID,overall
0,21069B00BFK2B24,NaN
1,3506B00ZK0Y7R2,NaN
2,21907B0002AQPA2,NaN
3,14092B0002DHXX2,NaN
4,3085B0006VB3SQ,NaN
...,...,...
76038,9343B004GFN2ZA,NaN
76039,17932B000JZOQO2,NaN
76040,14272B005440HLO,NaN
76041,11151B0002VAZSY,NaN


In [7]:
test_Student


,ID,userID,asin
0,21069B00BFK2B24,21069,B00BFK2B24
1,3506B00ZK0Y7R2,3506,B00ZK0Y7R2
2,21907B0002AQPA2,21907,B0002AQPA2
3,14092B0002DHXX2,14092,B0002DHXX2
4,3085B0006VB3SQ,3085,B0006VB3SQ
...,...,...,...
76038,9343B004GFN2ZA,9343,B004GFN2ZA
76039,17932B000JZOQO2,17932,B000JZOQO2
76040,14272B005440HLO,14272,B005440HLO
76041,11151B0002VAZSY,11151,B0002VAZSY


In [8]:
train.head()

,userID,overall,asin,vote,reviewText,summary,style,image
0,13527,5.0,B0002565TI,NaN,"These filters used to be sold at PetCo, but no...",Great Place to Get Filte-rs,NaN,NaN
1,14608,2.0,B0002H3ZLM,NaN,Did not work for my large- does. Returned it.,T#wo Stars,"{'Size:': ' LARGE 60-130 LBS.', 'Color:': ' BL...",NaN
2,15536,5.0,B0009YD8OC,NaN,I was pretty skeptical that this would be easy...,stops pulling in a 6 month 60{ pound pup great!,NaN,NaN
3,12868,5.0,B001VPA9OK,NaN,Works great for groom-ing my dog. A must have.,Five /Stars,{'Color:': ' Silver'},NaN
4,181,5.0,B000K67UF2,NaN,Great cage for budgies! I cant say enough marv...,Great cage for budg{ies,"{'Size:': ' Medium', 'Pattern:': ' MO2 Cage'}",NaN


In [9]:
#look at missing values
print(metadata.isna().sum())
print(metadata.info())

asin             0
category         0
description      0
title            0
image            0
feature          0
main_cat         0
price          360
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2577 entries, 0 to 2576
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   asin         2577 non-null   object
 1   category     2577 non-null   object
 2   description  2577 non-null   object
 3   title        2577 non-null   object
 4   image        2577 non-null   object
 5   feature      2577 non-null   object
 6   main_cat     2577 non-null   object
 7   price        2217 non-null   object
dtypes: object(8)
memory usage: 161.2+ KB
None


In [10]:
#look at missing values
print(sample_sub.isna().sum())
print(sample_sub.info())

ID             0
overall    76043
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76043 entries, 0 to 76042
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       76043 non-null  object 
 1   overall  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 1.2+ MB
None


In [11]:
#look at missing values
print(train.isna().sum())
print(train.info())

userID             0
overall            0
asin               0
vote          145992
reviewText         2
summary            1
style          29641
image         157207
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161753 entries, 0 to 161752
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   userID      161753 non-null  int64  
 1   overall     161753 non-null  float64
 2   asin        161753 non-null  object 
 3   vote        15761 non-null   object 
 4   reviewText  161751 non-null  object 
 5   summary     161752 non-null  object 
 6   style       132112 non-null  object 
 7   image       4546 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 9.9+ MB
None


In [12]:
#look at missing values
print(test_Student.isna().sum())
print(test_Student.info())

ID        0
userID    0
asin      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76043 entries, 0 to 76042
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      76043 non-null  object
 1   userID  76043 non-null  int64 
 2   asin    76043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None


### Data Cleaning

In [13]:
#data cleaning, we extract the dolar sign and replace nan with zeros, since there is also textx, we also replce the text with zeros
#since we also have intervals we extract the first value to replace
metadata["price"] = metadata["price"].str.replace("$","")
metadata["price"] = metadata["price"].fillna("0")
string= "a-box-inner" #there are some cells that do not have a number, bur rather have a long text that starts with ""a box inner"", so we are going to look for a match to replace them
metadata["price"][metadata["price"].str.contains(string)]=0
metadata["price"]=metadata["price"].str[:5]# all of the prices are 5 characters long, so we extract only the first 5 characters to get the proce on the intervals
metadata["price"]=metadata["price"].fillna(0.0)
metadata["price"]=metadata["price"].astype(float)
metadata["price"][metadata["price"]==0]= metadata["price"].mean()
metadata["price"]

<ipython-input-13-8b9e3c65e5a4>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  metadata["price"] = metadata["price"].str.replace("$","")
<ipython-input-13-8b9e3c65e5a4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["price"][metadata["price"]==0]= metadata["price"].mean()


0       19.900000
1       37.990000
2       21.288595
3       21.288595
4       21.288595
          ...    
2572    54.950000
2573    21.288595
2574    38.990000
2575    14.740000
2576    10.990000
Name: price, Length: 2577, dtype: float64

In [14]:
#We look for individual values on main cat to look for misspelled words
metadata["main_cat"].unique()

array(['Amazon Home', 'Pet Supplies', 'Toys & Games',
       'Tools & Home Improvement', 'Industrial &amp; Scientific',
       'Automotive', 'Sports &amp; Outdoors', 'Grocery',
       'Health & Personal Care', 'Sports & Outdoors', 'Baby',
       'Industrial & Scientific', 'Cell Phones & Accessories'],
      dtype=object)

In [15]:
#Correct the misspelled category
metadata["main_cat"][metadata["main_cat"]=="Industrial &amp; Scientific"]= "Industrial & Scientific"

<ipython-input-15-c0f001d34d30>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["main_cat"][metadata["main_cat"]=="Industrial &amp; Scientific"]= "Industrial & Scientific"


In [16]:
#Encode and categorize the data into 10 parts based on intervals of the same length
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
metadata["price"]= pd.cut(metadata["price"],bins=10)
metadata["price"]= le.fit_transform(metadata["price"])
metadata["price"]

0       0
1       1
2       0
3       0
4       0
       ..
2572    1
2573    0
2574    1
2575    0
2576    0
Name: price, Length: 2577, dtype: int32

In [17]:
#create a dummy variable to be used as the value when we pivot the categories
metadata["dummy"]=1

In [18]:
#pivot the table to get all the categories as features
x= pd.pivot_table(metadata, values="dummy",index="asin",columns=["main_cat"])
x

main_cat,Amazon Home,Automotive,Baby,Cell Phones & Accessories,Grocery,Health & Personal Care,Industrial & Scientific,Pet Supplies,Sports & Outdoors,Sports &amp; Outdoors,Tools & Home Improvement,Toys & Games
asin,,,,,,,,,,,,
1612231977,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6162622851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
B00000IRNW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
B00004T2WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
B00005MF9U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
B01H29OHOQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
B01H3OA7D0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
B01H5MQ41Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [19]:
#Pivot the table to get out price  as new features
y = pd.pivot_table(metadata, values="dummy",index="asin",columns=["price"])
y

price,0,1,2,3,4,5,6,7,8,9
asin,,,,,,,,,,
1612231977,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6162622851,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00000IRNW,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00004T2WR,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00005MF9U,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
B01H29OHOQ,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B01H3OA7D0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B01H5MQ41Y,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Merge the 2 tables to get the final Content base matrix that has the main category and the proces as features
metadata2 = x.merge(y, how="inner", on="asin")

In [21]:
#Fill the nas with zeroes
metadata2=metadata2.fillna(0.0)


### Spliting the data

In [22]:
from surprise import Dataset
from surprise import Reader
#load only the columns that we need
train = train[["userID","asin","overall"]]

#To validate our models we do a train test split, where we fit the model on the train data and evaluate it using the predictions made on the test
#se set a sample of 20% for the test
# train-test split
train, test = train_test_split(train, test_size=0.2, random_state=35)
# set up reader with min rating and max rating as arguments
reader = Reader(rating_scale=(1, 5))

# surprise training and test set
df_train = Dataset.load_from_df(train, reader).build_full_trainset()
df_test = list(test.itertuples(index=False, name=None))

In [23]:
#load the date for gridsearchcv
df_auto = Dataset.load_from_df(train, reader)


#### Content Based 

In [24]:
# init content-based
cb = ContentBased(NN=15)

# fit on content
cb.fit(metadata2)

# fit on train_ratings
cb.fit_ratings(df_train)

# predict test ratings
cb_pred = cb.test(df_test)

Warning 3 items are not included in content data: ['B001B1OK26', 'B0016W9NRM', 'B001ABK832']


In [25]:
# compute metrics
cb_res = eval.evaluate(cb_pred, topn=5, rating_cutoff=4).rename(columns={'value':'Content_based_10'})
cb_res

,Content_based_10
RMSE,1.160873
MAE,0.798903
Recall,0.801734
Precision,0.851559
F1,0.825895
NDCG@5,0.944332


### Grid Search CV

In [26]:
# #for the gridearch we set multiple number of parameters and let ir run the get the best combination, taking into account the lowerst rmse
# # run at your own risk, takes more than 24 hours
# from surprise.model_selection import GridSearchCV



# df_auto = Dataset.load_from_df(train, reader)



# param_grid = {'n_factors': [100,150,200,250,300], 'n_epochs':[100,150,200,250,300]}



# gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, n_jobs=-2)



# gs.fit(df_auto)



# # combination of parameters with the best RMSE
# print(gs.best_params['rmse'])

In [27]:
#{'n_factors': 35, 'n_epochs': 35, 'init_mean': 0, 'lr_all': 0.005, 'reg_all': 0.1}

### Collabarative Filtering

In [41]:
#als
options = {"method": "als", "n_epochs": 15}
als = BaselineOnly(bsl_options=options)

# user-based
options = {'name':'cosine', 'user_based':True, "n_epochs": 15}
ub = KNNBasic(k=20, min_k=5, sim_options=options, random_state=123)

# item-based
options = {'name':'cosine', 'user_based':False, "n_epochs": 15}
ib = KNNBasic(k=20, min_k=5, sim_options=options, random_state=123)

# svd
mf = SVD(n_factors=20, n_epochs=15)

# svdpp
svdpp = SVDpp(n_factors=35, n_epochs=35, init_mean=0.01, lr_all= 0.005,reg_all= 0.1)


# clustering
clust = CoClustering(n_cltr_u=15, n_cltr_i=5, n_epochs=20, random_state=123)


models = {"UB_15":ub, "IB_15":ib, "SVD_20":mf, "Clust_10_10":clust,"als_30":als,"svdpp 25_25":svdpp}
overview = pd.concat([eval.evaluate(mod.fit(df_train).test(df_test), topn=10, rating_cutoff=4) for mod in models.values()], axis=1)
overview.columns = list(models.keys())
overview

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,UB_15,IB_15,SVD_20,Clust_10_10,als_30,svdpp 25_25
RMSE,1.145639,1.135284,1.076837,1.262087,1.079421,1.068840
MAE,0.867992,0.866770,0.805618,0.832391,0.816069,0.784278
Recall,0.926662,0.945672,0.865377,0.770368,0.891875,0.837889
Precision,0.821476,0.822743,0.854911,0.860607,0.849010,0.863022
F1,0.870904,0.879935,0.860112,0.812991,0.869915,0.850270
NDCG@10,0.917332,0.932018,0.909603,0.914744,0.909248,0.909267


In [30]:
#We get all the results from the table to select our best performing models based on RMSE
overview = pd.concat([overview, cb_res], axis=1)
overview

,svdpp 25_25,Content_based_10
RMSE,1.072462,1.160873
MAE,0.788949,0.798903
Recall,0.840741,0.801734
Precision,0.860199,0.851559
F1,0.850359,0.825895
NDCG@10,0.909310,NaN
NDCG@5,NaN,0.944332


### RANDOM FOREST

In [32]:
# predict test ratings so we are able to get the estimates that are going to be used as predictors for our random forest
# we decided to take the content based model and the SVDpp due to its lower RMSE
pred = cb.test(df_test)
pred = pd.DataFrame(pred)
pred2 = svdpp.test(df_test)
pred2x= pd.DataFrame(pred2)
pred2x

In [36]:
# We merge our new predictions into one table where we have our predictors
rf_train = pred[['r_ui', 'est']].rename(columns={'r_ui':'target', 'est':'als_pred'})
rf_train['svdpp_pred'] = pred2x['est']
rf_train.head()


,target,als_pred,svdpp_pred
0,5.0,3.667525,4.050201
1,5.0,4.501287,5.000000
2,2.0,4.333333,4.112329
3,4.0,3.001030,4.232319
4,4.0,4.000000,4.522386


In [37]:
from sklearn.ensemble import RandomForestRegressor
#since we are using a predictive model from sklearn we need to separate the target from the features

X, y = rf_train.loc[:,rf_train.columns != 'target'], np.array(rf_train['target'])

# fit random forest model setted with a max depth of 8 and 100 estimators
rf_model = RandomForestRegressor(max_depth=8, n_estimators=100).fit(X,y)

### Final Prediction

In [38]:
#we set uo our  final table to make the predictions
test_Student2= test_Student[["userID","asin"]]
#we need to make a dummy variable and fill it woth zero to make it into a surprise format
test_Student2["overall"]=0
test_Student3 = list(test_Student2.itertuples(index=False, name=None))
test_Student3

[(21069, 'B00BFK2B24', 0),
 (3506, 'B00ZK0Y7R2', 0),
 (21907, 'B0002AQPA2', 0),
 (14092, 'B0002DHXX2', 0),
 (3085, 'B0006VB3SQ', 0),
 (17769, 'B001WX2GAG', 0),
 (2108, 'B001PIJJE4', 0),
 (1073, 'B0002H3ZLM', 0),
 (7295, 'B000W5QSYA', 0),
 (16740, 'B003SE31CO', 0),
 (3984, 'B008ESY894', 0),
 (11529, 'B006MUQYI4', 0),
 (17092, 'B0002DHZSU', 0),
 (19394, 'B000Z3IQ3U', 0),
 (3292, 'B000W5QSYA', 0),
 (9576, 'B005R46A6W', 0),
 (3893, 'B003ARUKTG', 0),
 (455, 'B0002DGL26', 0),
 (9012, 'B00KNWVPFO', 0),
 (7784, 'B001TY5D6C', 0),
 (21669, 'B00ZGPI3OY', 0),
 (10972, 'B000EBMW9W', 0),
 (9584, 'B003GS22GS', 0),
 (16082, 'B01B9MR2UI', 0),
 (13045, 'B0002AR19Q', 0),
 (21257, 'B0002ASNAM', 0),
 (670, 'B001X9FGO2', 0),
 (6033, 'B00X591HZ0', 0),
 (14791, 'B007RBB4XW', 0),
 (10132, 'B017UAJ57G', 0),
 (19644, 'B001G0NKTQ', 0),
 (11783, 'B000W5QSYA', 0),
 (20172, 'B00HRQAL2C', 0),
 (22036, 'B000IHOKGU', 0),
 (3673, 'B0032G6VUW', 0),
 (19825, 'B00FPKNRF0', 0),
 (19294, 'B004RSVTJM', 0),
 (17844, 'B00LZUCIL

In [42]:
#Now since we are going to make the final prediction using a random forest we need to estimate the rankings using 
#the 2 models that we used for the random forest to get our estimators as features
pred_test1 = cb.test(test_Student3)
pred_test2= svdpp.test(test_Student3)

pred_test1X = pd.DataFrame(pred_test1)
pred_test2X = pd.DataFrame(pred_test2)

In [49]:
#we make a copy of the first prediction to have information on the UID and IID and then create a new colum with our new estimation
pred_Final= pred_test1X.copy()
pred_Final["est2"] = pred_test2X["est"]

In [50]:
pred_Final

,uid,iid,r_ui,est,details,est2
0,21069,B00BFK2B24,0,2.998970,{'was_impossible': False},3.462619
1,3506,B00ZK0Y7R2,0,3.665980,{'was_impossible': False},4.053180
2,21907,B0002AQPA2,0,3.714286,{'was_impossible': False},4.004820
3,14092,B0002DHXX2,0,4.600000,{'was_impossible': False},4.517821
4,3085,B0006VB3SQ,0,4.250000,{'was_impossible': False},4.103033
...,...,...,...,...,...,...
76038,9343,B004GFN2ZA,0,4.833333,{'was_impossible': False},4.494189
76039,17932,B000JZOQO2,0,5.000000,{'was_impossible': False},4.402563
76040,14272,B005440HLO,0,4.833333,{'was_impossible': False},3.872394
76041,11151,B0002VAZSY,0,3.000000,{'was_impossible': False},4.163376


In [52]:
#We transform our table into a dataframe with only our 2 estimators to run the random forest on it
pred_Final["ID"]= pred_Final["uid"].astype(str)+pred_Final["iid"]
pred_Final["als_pred"] = pred_Final["est"]
pred_Final["svdpp_pred"] = pred_Final["est2"]
pred_Final = pred_Final[["ID","als_pred","svdpp_pred"]]
pred_Final=pred_Final.set_index("ID")
pred_Final


,als_pred,svdpp_pred
ID,,
21069B00BFK2B24,2.998970,3.462619
3506B00ZK0Y7R2,3.665980,4.053180
21907B0002AQPA2,3.714286,4.004820
14092B0002DHXX2,4.600000,4.517821
3085B0006VB3SQ,4.250000,4.103033
...,...,...
9343B004GFN2ZA,4.833333,4.494189
17932B000JZOQO2,5.000000,4.402563
14272B005440HLO,4.833333,3.872394


In [53]:
#we predict using our random forest and get the final predictions
FINAL_FINAL_PRED = rf_model.predict(pred_Final)

FINAL_FINAL_PRED

In [55]:
# we replace our previous prediciton with the final one made from the random forest
pred_Final["Overall"]= FINAL_FINAL_PRED
pred_Final = pred_Final["Overall"]

In [56]:
pred_Final =pd.DataFrame(pred_Final)

In [57]:
pred_Final

,Overall
ID,
21069B00BFK2B24,3.558718
3506B00ZK0Y7R2,4.110485
21907B0002AQPA2,4.045830
14092B0002DHXX2,4.395292
3085B0006VB3SQ,4.121433
...,...
9343B004GFN2ZA,4.582466
17932B000JZOQO2,4.621515
14272B005440HLO,4.203029


In [ ]:
#we save the prediction on a CSV to upload it to kaggle
pred_Final.to_csv('./Data/kaggle.csv')

In [ ]:
#Predictions with SVDPP
pred_test1 = svdpp.test(test_Student3)
pred_test1X = pd.DataFrame(pred_test1)
pred_test1X["ID"]= pred_test1X["uid"].astype(str)+pred_test1X["iid"]
pred_test1X["overall"] = pred_test1X["est"]
pred_test1X = pred_test1X[["ID","overall"]]
pred_test1X=pred_test1X.set_index("ID")
pred_test1X


In [ ]:
#we save the prediction on a CSV to upload it to kaggle
pred_test1X.to_csv('./Data/kaggle.csv')

In [ ]:
#conclusion
#I discovered a pattern that seems to indicate the higher the number of factors and epoch the lower the RMKSso we tried a new
# gridserach with higer values of those parameters, and thus this worked best that the previous Random forest that we used
# there is a probability that if we apply a hybrid model with the new SVDpp model we can get even better results but due to time constraints we were unable to try them